In [2]:
import keras
import random
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.random.seed(42) # NumPy
random.seed(42) # Python
tf.random.set_seed(42) # Tensorflow

In [3]:
import optuna

In [4]:
train_data = pd.read_pickle('final_train_data.pkl')

In [5]:
# Feature selection
X = train_data.drop(['isFraud'], axis=1)
y = train_data['isFraud']
# split the data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [6]:
from imblearn.over_sampling import SMOTE

In [7]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
print(X_train_smote.shape)
print(y_train_smote.shape)

(911646, 171)
(911646,)


In [10]:
import optuna
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def objective(trial):
    # Define the hyperparameters to optimize
    num_layers = trial.suggest_int('num_layers', 2, 4)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])
    layer_units = []
    
    # Define the number of neurons in each layer
    for i in range(num_layers):
        layer_units.append(trial.suggest_int(f'layer_{i}_units', 32, 512, log=True))
    
    # Define the model architecture
    model = Sequential()
    for i, units in enumerate(layer_units):
        if i == 0:
            model.add(Dense(units, activation='relu', input_shape=[X_train_smote.shape[1]]))
        else:
            model.add(Dense(units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # Define optimizer with the suggested learning rate
    if optimizer == 'adam':
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)

    auc_metric = tf.keras.metrics.AUC(name='auc')

    
    # Compile the model with the specified optimizer
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[auc_metric])
    
    # Define callbacks
    # reduce_lr = ReduceLROnPlateau(patience=3, monitor='val_auc', mode='max', patience=5)
    early_stop = EarlyStopping(patience=5, monitor='val_auc', restore_best_weights=True)
    
    # Train the model
    history = model.fit(X_train_smote, y_train_smote, batch_size=512, validation_data=(X_val, y_val), epochs=15, callbacks=[early_stop])
    
    # Predict probabilities for validation set
    y_pred_proba = model.predict(X_val)
    
    # Calculate AUC
    auc = roc_auc_score(y_val, y_pred_proba)
    
    return auc
    
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)


best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2023-06-01 17:03:56,183] A new study created in memory with name: no-name-6fa8dc94-6d15-4473-a799-97a9f4f8a290


Epoch 1/15
1781/1781 [==============================] - 80s 44ms/step - loss: 1.0985 - auc: 0.5023 - val_loss: 0.6991 - val_auc: 0.5000
Epoch 2/15
1781/1781 [==============================] - 20s 11ms/step - loss: 0.6932 - auc: 0.5013 - val_loss: 0.6928 - val_auc: 0.5000
Epoch 3/15
1781/1781 [==============================] - 20s 11ms/step - loss: 0.6932 - auc: 0.5010 - val_loss: 0.6931 - val_auc: 0.5001
Epoch 4/15
1781/1781 [==============================] - 21s 12ms/step - loss: 0.6932 - auc: 0.5000 - val_loss: 0.6881 - val_auc: 0.5000
Epoch 5/15
1781/1781 [==============================] - 18s 10ms/step - loss: 0.6934 - auc: 0.5004 - val_loss: 0.6958 - val_auc: 0.5000
Epoch 6/15
1781/1781 [==============================] - 20s 11ms/step - loss: 0.6932 - auc: 0.4995 - val_loss: 0.6960 - val_auc: 0.5000
Epoch 7/15
1781/1781 [==============================] - 20s 11ms/step - loss: 0.6934 - auc: 0.4998 - val_loss: 0.6937 - val_auc: 0.5000
Epoch 8/15
3691/3691 [==========================

[I 2023-06-01 17:18:42,668] Trial 0 finished with value: 0.5001233349777997 and parameters: {'num_layers': 5, 'dropout_rate': 0.40236986019862775, 'learning_rate': 0.0009587831378323827, 'optimizer': 'adam', 'layer_0_units': 87, 'layer_1_units': 368, 'layer_2_units': 51, 'layer_3_units': 136, 'layer_4_units': 161}. Best is trial 0 with value: 0.5001233349777997.


Epoch 1/15
1781/1781 [==============================] - 75s 42ms/step - loss: 6.9655 - auc: 0.5534 - val_loss: 0.7970 - val_auc: 0.5654
Epoch 2/15
1781/1781 [==============================] - 13s 7ms/step - loss: 0.6824 - auc: 0.5775 - val_loss: 0.7239 - val_auc: 0.6675
Epoch 3/15
1781/1781 [==============================] - 14s 8ms/step - loss: 0.6715 - auc: 0.6206 - val_loss: 0.7417 - val_auc: 0.6798
Epoch 4/15
1781/1781 [==============================] - 13s 7ms/step - loss: 0.6631 - auc: 0.6396 - val_loss: 0.7477 - val_auc: 0.6829
Epoch 5/15
1781/1781 [==============================] - 13s 7ms/step - loss: 0.6594 - auc: 0.6439 - val_loss: 0.7216 - val_auc: 0.6833
Epoch 6/15
1781/1781 [==============================] - 15s 8ms/step - loss: 0.6555 - auc: 0.6489 - val_loss: 0.6655 - val_auc: 0.6809
Epoch 7/15
1781/1781 [==============================] - 13s 7ms/step - loss: 0.6531 - auc: 0.6502 - val_loss: 0.5900 - val_auc: 0.6501
Epoch 8/15
1781/1781 [==============================] 

[I 2023-06-01 17:33:39,839] Trial 1 finished with value: 0.6895768822372708 and parameters: {'num_layers': 1, 'dropout_rate': 0.4945579666361451, 'learning_rate': 0.0005793997326872113, 'optimizer': 'sgd', 'layer_0_units': 203}. Best is trial 1 with value: 0.6895768822372708.


Epoch 1/15
1781/1781 [==============================] - 80s 44ms/step - loss: 3.7300 - auc: 0.5288 - val_loss: 0.7620 - val_auc: 0.5516
Epoch 2/15
1781/1781 [==============================] - 16s 9ms/step - loss: 0.9107 - auc: 0.5279 - val_loss: 0.7371 - val_auc: 0.5436
Epoch 3/15
1781/1781 [==============================] - 14s 8ms/step - loss: 0.8221 - auc: 0.5288 - val_loss: 0.7231 - val_auc: 0.5381
Epoch 4/15
1781/1781 [==============================] - 14s 8ms/step - loss: 0.7815 - auc: 0.5275 - val_loss: 0.7115 - val_auc: 0.5349
Epoch 5/15
1781/1781 [==============================] - 14s 8ms/step - loss: 0.7540 - auc: 0.5259 - val_loss: 0.7103 - val_auc: 0.5374
Epoch 6/15
3691/3691 [==============================] - 5s 1ms/step


[I 2023-06-01 17:46:54,779] Trial 2 finished with value: 0.5521666577604742 and parameters: {'num_layers': 2, 'dropout_rate': 0.09986610131263085, 'learning_rate': 1.4554369503050305e-05, 'optimizer': 'sgd', 'layer_0_units': 32, 'layer_1_units': 169}. Best is trial 1 with value: 0.6895768822372708.


: 

: 